## XGBoost+LR模型

In [3]:
from sklearn.tree import DecisionTreeClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [4]:
dpath = "./data/"

In [5]:
train_merge = pd.read_csv(dpath+"train_merge.csv")

In [6]:
train_merge.head()

,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator,clickTime_day,clickTime_hour,...,adID,camgaignID,advertiserID,appID,appPlatform,appCategory,app_categories_first_class,app_categories_second_class,sitesetID,positionType
0,0,170000,NaN,3089,2798058,293,1,1,17,0,...,1321,83,10,434,1,108,1,8,0,3
1,0,180028,NaN,3089,1683269,293,2,3,18,0,...,1321,83,10,434,1,108,1,8,0,3
2,0,191045,NaN,3089,240899,293,1,2,19,1,...,1321,83,10,434,1,108,1,8,0,3
3,0,182300,NaN,2230,2177495,293,2,1,18,3,...,2841,83,10,434,1,108,1,8,0,3
4,0,200113,NaN,2230,417301,293,2,2,20,0,...,2841,83,10,434,1,108,1,8,0,3


In [7]:
X_train = train_merge.loc[:,['creativeID',
       'positionID', 'connectionType', 'telecomsOperator', 'clickTime_day',
       'clickTime_hour', 'age', 'gender', 'education', 'marriageStatus',
       'haveBaby', 'residence', 'hometown_province',
       'hometown_city', 'residence_province', 'residence_city', 'adID',
       'camgaignID', 'advertiserID', 'appID', 'appPlatform',
       'app_categories_first_class', 'app_categories_second_class',
       'sitesetID', 'positionType']]
X_train = np.array(X_train.values,dtype='int32')
y_train = train_merge.iloc[:,0].values

In [16]:
%%time
params = {'learning_rate': 0.05,
          'n_estimators': 20, 
          'max_depth': 5,
          'subsample': 0.6,
          'colsample_bytree':0.5,
          'nthread': 4,
         }
xgb = XGBClassifier(**params)
xgb.fit(X_train,y_train)

Wall time: 32.8 s


In [17]:
%%time
X_train_leaf = xgb.apply(X_train)

Wall time: 3.24 s


In [18]:
X_train_leaf.shape

(3749528, 20)

In [19]:
%%time
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(X_train_leaf)

D:\Program Files\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Wall time: 7.59 s


In [20]:
%%time
X_train_enc = enc.transform(X_train_leaf)

Wall time: 4.04 s


In [21]:
X_train_enc.shape

(3749528, 501)

In [23]:
type(X_train_enc)

scipy.sparse.csr.csr_matrix

In [24]:
import scipy.io as sio
sio.mmwrite("./data/X_train_enc", X_train_enc)

In [30]:
pickle.dump(y_train,open("./data/y_train.pkl","wb"))

In [ ]:
sio.mmread("./data/X_train_enc").todense()

In [25]:
%%time
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(n_jobs=4, C=0.1, penalty='l1')
lr.fit(X_train_enc,y_train)

D:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 24min 14s


In [26]:
y_proba = lr.predict_proba(X_train_enc)
auc = roc_auc_score(y_train,y_proba[:,1])
print(auc)

0.7528303683897513


In [27]:
pickle.dump(y_proba,open("./result/xgb_lr_pred.pkl","wb"))

In [28]:
type(y_train)

numpy.ndarray